In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
import shutil
import yaml
import re
import html

In [2]:
db = os.environ.get('PSQL_DATABASE')
host = 'localhost'
user ='user'
port = '5432'
psql_engine = f'postgresql://{user}:@{host}:{port}/{db}'

In [3]:
with open("get_poll_list.sql", "r") as f:
    poll_list_slq = f.read()

In [4]:
df = pd.read_sql(poll_list_slq, psql_engine)
poll_list = {
    value['topic_id'] : {
        'question' : value['poll_title'],
        'max_votes' : value['max_votes']
    }
    for index, value in df.T.to_dict().items()
}

In [5]:
with open("get_poll_results.sql", "r") as f:
    poll_results_template = f.read()

In [6]:
for k in poll_list.keys():
    poll_results_sql = poll_results_template.format(topic_id = k)
    df = pd.read_sql(poll_results_sql, psql_engine)
    poll_list[k]['answers'] =  [i for i in df.T.to_dict().values()]

In [7]:
file_path = os.path.join('..', '_tests', '_data')
with open(os.path.join(file_path, 'polls.yml'), 'w') as f:
    f.write(yaml.dump(poll_list))